In [8]:
import torch
import sys
import time

### Training model

#### Implementing custom dataset

In [9]:
from dataset import LMDBDataset 

sys.path.append('./config')
from train import train_start_index, train_end_index, data_path, batch_size, val_start_index, val_end_index, is_import_model, model_path, num_epochs, save_path

In [10]:
def collate_fn(batch):
    return tuple(zip(*batch))


train_dataset = LMDBDataset(data_path['train'],train_start_index,train_end_index)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True,collate_fn=collate_fn, num_workers=4)

val_dataset = LMDBDataset(data_path['val'],val_start_index,val_end_index)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=True,collate_fn=collate_fn, num_workers=4)

#### Setting Model

In [11]:
from model import FasterRCNNResNet50 

In [12]:
model =  torch.load(model_path) if is_import_model else FasterRCNNResNet50(num_classes=14)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

/home/brandonrafaellovelyno/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


FasterRCNNResNet50(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=1e-05)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=1e-05)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=1e-05)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
 

In [13]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

#### Looping train

In [14]:
for epoch in range(num_epochs):
    model.train()     
       
    running_loss = 0.0
    for i, (images, targets) in enumerate(train_dataloader):
        
        if i % 100 == 0:
            start_time = time.time()

        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        optimizer.zero_grad()

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        losses.backward()
        optimizer.step()

        running_loss += losses.item()

        if i % 100 == 0:
            print("Estimated all training time in minutes: ", (time.time() - start_time) * (len(train_dataloader)) * num_epochs / 60)
    
    avg_train_loss = running_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Training Loss: {avg_train_loss}")        

    lr_scheduler.step()


    val_running_loss = 0.0
    with torch.no_grad():
        for i, (images, targets) in enumerate(val_dataloader):

            if i % 100 == 0:
                start_time = time.time()

            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            val_running_loss += losses.item()

            if i % 100 == 0:
                print("Estimated all validation time in minutes: ", (time.time() - start_time) * (len(val_dataloader)) * num_epochs / 60)


    avg_val_loss = val_running_loss / len(val_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Validation Loss: {avg_val_loss}")
    
print("Training complete!")


Estimated all training time in minutes:  386.8650701165199


KeyboardInterrupt: 

#### Saving trained model

In [8]:
torch.save(model.state_dict(), save_path)